# Part 1

In [1]:
import numpy as np
import pandas as pd
from math import e
from scipy.optimize import minimize
from scipy.stats import gamma
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.optimize import minimize
from scipy.special import factorial
import math

In [2]:
df = pd.read_csv('candy.csv')

In [3]:
df.head()

,Packs,People
0,0,102
1,1,54
2,2,49
3,3,62
4,4,44


#### The Poisson Model

In [4]:
#1. we name the arrays that will go into the function
a = np.array(df[['People']])
b = np.array(df[['Packs']])
lmbda = 1

#2.we define the Poisson Regression Model
def PLL(lmbda,a,b):
    #variable c will store the cumulative log-likelihood
    c = 0 
    #the for loop below will iterate throuh each row of the dataset
    for i in range(len(a)): 
        c += a[i]*np.log(poisson.pmf(b[i], lmbda))
    return (-1)*c 

#3.we use the minimize function to optimize the parameter
#(in this case, LAMBDA) to maximize the log likelihood
solpm1 = minimize(
    PLL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

#4. we print the parameters and final log likelihood value
final_lmbda = solpm1.x[0]
ll_pm1 = solpm1.fun[0]
print("Lambda:{}".format(final_lmbda))
print("Log likelihood:{}".format(ll_pm1*(-1)))

Lambda:3.9912283512077944
Log likelihood:-1544.9963904489719


#### The NBD Model

In [5]:
#1. define the NBD model function
def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t)))*NBD(params,k-1,t)

#2. define the model that will calculate the cumulative log likelihood of the NBD model
def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*np.log(NBD(params,b[i],t))
    return (-1)*nbd

#3. run the NBDLL model through the minimize function
#this will optimize the parameters alpha and n in order to maximize the log likelihood
alpha = 1
n = 1
t = 1
solnbd1 = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
alpha = solnbd1.x[0]
n = solnbd1.x[1]
ll_nbd1 = solnbd1.fun[0]
print("Alpha:{}".format(alpha))
print("n:{}".format(n))
print("Log likelihood:{}".format(ll_nbd1*(-1)))

Alpha:0.2499605034092882
n:0.9976364542507823
Log likelihood:-1140.0237462459445


#### The Zero Inflated NBD Model

In [6]:
candy=pd.read_csv('candy.csv')
k = np.array(candy[['Packs']]) #define the first array that will go through the PLL function
b = np.array(candy[['People']]) #define the second array that will go through PLL

def NBD(params,k):#this function will calculate NBD
    alpha,n,pi = params #three parameters
    if k==0:
        return ((alpha/(alpha+1))**n)
    else:
        return (((n+k-1)/(k*(alpha+1)))*NBD(params,k-1))
    
def ZINBD(params,k):#this function will calculate the ZERO INFLATED NBD by utilizing the NBD function
    alpha,n,pi= params
    if k==0:
        return pi +(1-pi)*NBD(params,k)
    else:
        return (1-pi)*NBD(params,k)
    
def ZINBDLL(params,k,b): #this function will calculate the log likelihood of a ZI NBD by utilizing the ZINBD function
    alpha,n,pi=params
    zinbd = 0
    for i in range(len(b)):
        zinbd += b[i]*np.log(ZINBD(params,k[i]))
    return (-1)*zinbd

alpha = 1 # 000001 -inf
n = 1 # 000001 -inf
pi =0.5 # 000001 -0.99999 
solzinbd = minimize(
    ZINBDLL,
    args = (k,b),
    x0 = np.array((1,1.5,0.00000001)),
    bounds=[(0.000001,None),(0.000001,None),(0.000001,0.999999)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

solzinbd

      fun: array([1136.16564083])
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00311502,  0.00070486, -0.00100044])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 68
      nit: 14
     njev: 17
   status: 0
  success: True
        x: array([0.33418767, 1.50392276, 0.11310436])

#### The Finite Mixture Models for 2,3,4 segments

#### 2-segment

In [7]:
def two_segment(params,a,b):
    pi,lambda1,lambda2 = params
    ll = 0
    for i in range(len(b)):
        ll += a[i]*np.log(pi*(poisson.pmf(b[i],lambda1)) + (1-pi)*(poisson.pmf(b[i],lambda2)))
    return (-1)*ll

a = np.array(df[['People']])
b = np.array(df[['Packs']])
sol2segment = minimize(
    two_segment,
    args = (a,b),
    x0 = np.array((1,1,1)),
    bounds=[(0.000001,0.999999),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
print("lambda1 value for two-segment {}".format(sol2segment.x[2]))
print("lambda2 value for two-segment {}".format(sol2segment.x[1]))
print("pi1 value for two-segment {}".format(sol2segment.x[0]))
print("pi2(1-pi1) value for two-segment {}".format(1-sol2segment.x[0]))
print("Log likelihood for 2-segment finite mixture model{}".format(-1*sol2segment.fun[0]))

lambda1 value for two-segment 1.802152486641123
lambda2 value for two-segment 9.120665766225097
pi1 value for two-segment 0.2991217614346511
pi2(1-pi1) value for two-segment 0.7008782385653489
Log likelihood for 2-segment finite mixture model-1188.8328272199415


#### 3-segment

In [8]:
def three_segment(params,a,b):
    theta1,theta2,lambda1,lambda2,lambda3 = params
    pi1 = (e**theta1)/((e**theta1)+(e**theta2)+1) # calculating pie values(pie1,pie2,pie3)
    pi2 = (e**theta2)/((e**theta1)+(e**theta2)+1)
    pi3 = (1)/((e**theta1)+(e**theta2)+1)
    ll = 0
    for i in range(len(b)):
        ll += a[i]*np.log(pi1*(poisson.pmf(b[i],lambda1)) + pi2*(poisson.pmf(b[i],lambda2)) + pi3*(poisson.pmf(b[i],lambda3))) # as per the formula given in the slides 
    return (-1)*ll  


a = np.array(df[['People']])
b = np.array(df[['Packs']])
sol3segment = minimize(
    three_segment,
    args = (a,b),
    x0 = np.array((1,2,1,1,1)),
    bounds=[(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

theta1 = sol3segment.x[0]
theta2 = sol3segment.x[1]
lambda1 = sol3segment.x[2]
lambda2 = sol3segment.x[3]
lambda3 = sol3segment.x[4]
print("lambda1 value for three-segment {}".format(lambda1))
print("lambda2 value for three-segment {}".format(lambda2))
print("lambda3 value for three-segment {}".format(lambda3))
print("theta1 value for three-segment {}".format(theta1))
print("theta2 value for three-segment {}".format(theta2))
print("theta3 value for three-segment 0.0")
print("Log likelihood for 3-segment finite mixture model {}".format(-1*sol3segment.fun[0]))

lambda1 value for three-segment 3.483322340334329
lambda2 value for three-segment 11.215816773362894
lambda3 value for three-segment 0.29055366607878474
theta1 value for three-segment 0.6744315631930806
theta2 value for three-segment -0.43040790312205945
theta3 value for three-segment 0.0
Log likelihood for 3-segment finite mixture model -1132.0429842751907


#### 4-segment

In [9]:
def four_segment(params,a,b):
    theta1,theta2,theta3,lambda1,lambda2,lambda3,lambda4 = params
    pie1 = (e**theta1)/((e**theta1)+(e**theta2)+(e**theta3)+1) # calculating pie values(pie1,pie2,pie3)
    pie2 = (e**theta2)/((e**theta1)+(e**theta2)+(e**theta3)+1)
    pie3 = (e**theta3)/((e**theta1)+(e**theta2)+(e**theta3)+1)
    pie4 = (1)/((e**theta1)+(e**theta2)+(e**theta3)+1)
    ll = 0
    for i in range(len(b)):
        ll += a[i]*np.log(pie1*(poisson.pmf(b[i],lambda1)) + pie2*(poisson.pmf(b[i],lambda2)) + pie3*(poisson.pmf(b[i],lambda3))+pie4*(poisson.pmf(b[i],lambda4))) # as per the formula given in the slides 
    return (-1)*ll  

a = np.array(df[['People']])
b = np.array(df[['Packs']])
sol4segment = minimize(
    four_segment,
    args = (a,b),
    x0 = np.array((1,2,3,1,1,1,1)),
    bounds=[(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
#print(three_segment(np.array((0.430,1.105,0.291,3.483,11.216)),a,b))
theta1 = sol4segment.x[0]
theta2 = sol4segment.x[1]
theta3 = sol4segment.x[2]
lambda1 = sol4segment.x[3]
lambda2 = sol4segment.x[4]
lambda3 = sol4segment.x[5]
lambda4 = sol4segment.x[6]
print("lambda1 value for four-segment {}".format(lambda1))
print("lambda2 value for four-segment {}".format(lambda2))
print("lambda4 value for four-segment {}".format(lambda3))
print("lambda4 value for four-segment {}".format(lambda4))
print("theta1 value for four-segment {}".format(theta1))
print("theta2 value for four-segment {}".format(theta2))
print("theta3 value for four-segment {}".format(theta3))
print("theta4 value for four-segment 0.0")
print("Log likelihood for 4-segment finite mixture model {}".format(-1*sol4segment.fun[0]))

lambda1 value for four-segment 7.417654014502218
lambda2 value for four-segment 0.2047757185785757
lambda4 value for four-segment 12.872409491251577
lambda4 value for four-segment 3.0019275641803396
theta1 value for four-segment -1.2001167375208144
theta2 value for four-segment -0.7220301738582039
theta3 value for four-segment -1.5980245259938597
theta4 value for four-segment 0.0
Log likelihood for 4-segment finite mixture model -1130.0705913673658


2-segment prediction

In [10]:
#2 segments
lambda2 = 1.802152486641123
lambda1 = 9.120665766225097
pie1 = 0.2991217614346511
pie2 = 0.7008782385653489

prob_s1_5 = (pie1*poisson.pmf(5,lambda1))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2))
prob_s2_5 = (pie2*poisson.pmf(5,lambda2))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2))
print("Expected purchases next 8 weeks if past week = 5 packs is {}".format(8*(lambda1*(prob_s1_5) + lambda2*(prob_s2_5))))
maxi = max(prob_s1_5,prob_s2_5)
print("Expected purchases next 8 weeks if past week = 5 packs, if customer is classified into segment2(since p(s=1,x=5) < p(s=2,x=5)) is {}".format(8*lambda2))

Expected purchases next 8 weeks if past week = 5 packs is 42.781266102377096
Expected purchases next 8 weeks if past week = 5 packs, if customer is classified into segment2(since p(s=1,x=5) < p(s=2,x=5)) is 14.417219893128983


3-segment prediction

In [11]:
#3segments
lambda1 = 3.483321426064036
lambda2 = 11.2158213224157
lambda3 = 0.29055258688349966
theta1 = 0.6744334195584446
theta2 = -0.43040650590390545
theta3 = 0.0
pie1 = e**(theta1) / ((e**theta1) + (e**theta2) + (e**theta3))
pie2 = e**(theta2) / ((e**theta1) + (e**theta2) + (e**theta3))
pie3 = e**(theta3) / ((e**theta1) + (e**theta2) + (e**theta3))

prob_s1_5 = (pie1*poisson.pmf(5,lambda1))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3))
prob_s2_5 = (pie2*poisson.pmf(5,lambda2))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3))
prob_s3_5 = (pie3*poisson.pmf(5,lambda3))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3))
print("Expected purchases next 8 weeks if past week = 5 packs is {}".format(8*(lambda1*(prob_s1_5) + lambda2*(prob_s2_5) + lambda3*(prob_s3_5))))
maxi = max(prob_s1_5,prob_s2_5,prob_s3_5)
print("Expected purchases next 8 weeks if past week = 5 packs, if customer is classified into segment1(since prob(s=2,x=5) & prob(s=3,x=5) < prob(s=1,x=5)) is {}".format(8*lambda1))

Expected purchases next 8 weeks if past week = 5 packs is 30.825233285033676
Expected purchases next 8 weeks if past week = 5 packs, if customer is classified into segment1(since prob(s=2,x=5) & prob(s=3,x=5) < prob(s=1,x=5)) is 27.86657140851229


4-segment prediction

In [12]:
#4segment
lambda1 = 7.417654014502218
lambda2 = 0.2047757185785757
lambda3 = 12.872409491251577
lambda4 = 3.0019275641803396
theta1 = -1.2001167375208144
theta2 = -0.7220301738582039
theta3 = -1.5980245259938597
theta4 = 0.0
pie1 = e**(theta1) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))
pie2 = e**(theta2) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))
pie3 = e**(theta3) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))
pie4 = e**(theta4) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))


prob_s1_5 = (pie1*poisson.pmf(5,lambda1))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3) + pie4*poisson.pmf(5,lambda4))
prob_s2_5 = (pie2*poisson.pmf(5,lambda2))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3) + pie4*poisson.pmf(5,lambda4))
prob_s3_5 = (pie3*poisson.pmf(5,lambda3))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3) + pie4*poisson.pmf(5,lambda4))
prob_s4_5 = (pie4*poisson.pmf(5,lambda4))/(pie1*poisson.pmf(5,lambda1) + pie2*poisson.pmf(5,lambda2) + pie3*poisson.pmf(5,lambda3) + pie4*poisson.pmf(5,lambda4))
print("Expected purchases next 8 weeks if past week = 5 packs is {}".format(8*(lambda1*(prob_s1_5) + lambda2*(prob_s2_5) + lambda3*(prob_s3_5) + lambda4*(prob_s4_5))))
maxi = max(prob_s1_5,prob_s2_5,prob_s3_5,prob_s4_5)
print("Expected purchases next 8 weeks if past week = 5 packs is, if customer is classified into segment1(since p(s=2,x=5) & p(s=3,x=5) & p(s=1,x=5) < p(s=4,x=5)) is {}".format(8*lambda4))

Expected purchases next 8 weeks if past week = 5 packs is 33.67181197102352
Expected purchases next 8 weeks if past week = 5 packs is, if customer is classified into segment1(since p(s=2,x=5) & p(s=3,x=5) & p(s=1,x=5) < p(s=4,x=5)) is 24.015420513442717


2-segment prediction

In [13]:
#2segment
lambda2 = 1.802152486641123
lambda1 = 9.120665766225097
pie1 = 0.2991217614346511
pie2 = 0.7008782385653489

prob_s1_9 = (pie1*poisson.pmf(9,lambda1))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2))
prob_s2_9 = (pie2*poisson.pmf(9,lambda2))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2))
print("Expected purchases next 8 weeks if past week = 9 packs is {}".format(8*(lambda1*(prob_s1_9) + lambda2*(prob_s2_9))))
maxi = max(prob_s1_9,prob_s2_9)
print("Expected purchases next 8 weeks if past week = 9 packs is, if customer is classified into segment2(since p(s=2,x=9) < p(s=1,x=9)) is {}".format(8*lambda1))

Expected purchases next 8 weeks if past week = 5 packs is 72.87051083131314
Expected purchases next 8 weeks if past week = 5 packs is, if customer is classified into segment2(since p(s=2,x=9) < p(s=1,x=9)) is 72.96532612980077


3-segment prediction

In [14]:
lambda1 = 3.483321426064036
lambda2 = 11.2158213224157
lambda3 = 0.29055258688349966
theta1 = 0.6744334195584446
theta2 = -0.43040650590390545
theta3 = 0.0
pie1 = e**(theta1) / ((e**theta1) + (e**theta2) + (e**theta3))
pie2 = e**(theta2) / ((e**theta1) + (e**theta2) + (e**theta3))
pie3 = e**(theta3) / ((e**theta1) + (e**theta2) + (e**theta3))

prob_s1_9 = (pie1*poisson.pmf(9,lambda1))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3))
prob_s2_9 = (pie2*poisson.pmf(9,lambda2))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3))
prob_s3_9 = (pie3*poisson.pmf(9,lambda3))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3))
print("Expected purchases next 8 weeks if past week = 9 packs is {}".format(8*(lambda1*(prob_s1_9) + lambda2*(prob_s2_9) + lambda3*(prob_s3_9))))
maxi = max(prob_s1_9,prob_s2_9,prob_s3_9)
print("Expected purchases next 8 weeks if past week = 9 packs is, if customer is classified into segment1(since prob(s=1,x=9) & prob(s=3,x=9) < prob(s=2,x=9)) is {}".format(8*lambda2))

Expected purchases next 8 weeks if past week = 5 packs is 80.06350112030917
Expected purchases next 8 weeks if past week = 5 packs is, if customer is classified into segment1(since prob(s=1,x=9) & prob(s=3,x=9) < prob(s=2,x=9)) is 89.7265705793256


4-segment prediction

In [15]:
lambda1 = 7.417654014502218
lambda2 = 0.2047757185785757
lambda3 = 12.872409491251577
lambda4 = 3.0019275641803396
theta1 = -1.2001167375208144
theta2 = -0.7220301738582039
theta3 = -1.5980245259938597
theta4 = 0.0
pie1 = e**(theta1) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))
pie2 = e**(theta2) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))
pie3 = e**(theta3) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))
pie4 = e**(theta4) / ((e**theta1) + (e**theta2) + (e**theta3) + (e**theta4))


prob_s1_9 = (pie1*poisson.pmf(9,lambda1))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3) + pie4*poisson.pmf(9,lambda4))
prob_s2_9 = (pie2*poisson.pmf(9,lambda2))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3) + pie4*poisson.pmf(9,lambda4))
prob_s3_9 = (pie3*poisson.pmf(9,lambda3))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3) + pie4*poisson.pmf(9,lambda4))
prob_s4_9 = (pie4*poisson.pmf(9,lambda4))/(pie1*poisson.pmf(9,lambda1) + pie2*poisson.pmf(9,lambda2) + pie3*poisson.pmf(9,lambda3) + pie4*poisson.pmf(9,lambda4))
print("Expected purchases next 8 weeks if past week = 9 packs is {}".format(8*(lambda1*(prob_s1_9) + lambda2*(prob_s2_9) + lambda3*(prob_s3_9) + lambda4*(prob_s4_9))))
maxi = max(prob_s1_5,prob_s2_5,prob_s3_5,prob_s4_5)
print("Expected purchases next 8 weeks if past week = 9 packs is, if customer is classified into segment1(since p(s=2,x=9) & p(s=3,x=9) & p(s=4,x=9) < p(s=1,x=9)) is {}".format(8*lambda1))

Expected purchases next 8 weeks if past week = 5 packs is 69.4515852214773
Expected purchases next 8 weeks if past week = 5 packs is, if customer is classified into segment1(since p(s=2,x=9) & p(s=3,x=9) & p(s=4,x=9) < p(s=1,x=9)) is 59.34123211601774


# Part 2

In [16]:
df=pd.read_csv('articles.csv')
df.head()

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26


In [17]:
df=pd.read_csv('articles.csv')
k=np.array(df[['articles']])
a=np.array(df[['female']])
b=np.array(df[['married']])
c=np.array(df[['kids']])
d=np.array(df[['prestige']])
f=np.array(df[['menpubs']])

def PR(params,a,b,c,d,f,k):
    lmbda0, beta1, beta2, beta3, beta4, beta5 = params
    p = 0
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        lmbda= lmbda0*math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i]+beta5*f[i])
        fact = float(math.factorial(float(k[i])))## take the log before 
        pt1 = pt1 + k[i]*np.log(lmbda)
        pt2 = pt2 + lmbda
        pt3 = pt3 + np.log(fact)
    p = pt1-pt2-pt3
    return (-1)*p
lmbda0=0.0001
beta1=0.0001
beta2=0.0001
beta3=0.0001
beta4=0.0001
beta5=0.0001
params=lmbda0,beta1,beta2,beta3,beta4,beta5
solPR1 = minimize(
    PR,
    args = (a,b,c,d,f,k),
    x0 = np.array(params),
    bounds=[(0.00001, None), (None,None), (None,None), (None,None), (None,None), (None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

lmbda0 = solPR1.x[0]
beta1=solPR1.x[1]
beta2=solPR1.x[2]
beta3=solPR1.x[3]
beta4=solPR1.x[4]
beta4=solPR1.x[5]
ll_PR1=solPR1.fun[0]
print("Lambda0: {}".format(lmbda0))
print("Beta1: {}".format(beta1))
print("Beta2: {}".format(beta2))
print("Beta3: {}".format(beta3))
print("Beta4: {}".format(beta4))
print("Beta5: {}".format(beta5))
print("Log Likelihood: {}".format(ll_PR1*(-1)))

Lambda0: 1.3569756192417102
Beta1: -0.2239520330683631
Beta2: 0.1549662615389069
Beta3: -0.18508761154450679
Beta4: 0.02553308422651582
Beta5: 0.0001
Log Likelihood: -1651.0565708515671


In [18]:
df=pd.read_csv('articles.csv')

a=np.array(df[['female']])
b=np.array(df[['married']])
c=np.array(df[['kids']])
d=np.array(df[['prestige']])
f=np.array(df[['menpubs']])
k=np.array(df[['articles']])

def NBDR(params,a,b,c,d,f,k):
    alpha, n, beta1, beta2, beta3, beta4, beta5 = params
    p = 0
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        pt1 = np.log(math.gamma(n+int(k[i]))) -(np.log(math.gamma(n)) + np.log(factorial(int(k[i]))))
        denom = math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i]+beta5*f[i])
        pt2 = n*(np.log(alpha) - np.log(alpha+denom))
        pt3 = int(k[i])*(np.log(denom) - np.log(alpha+denom))
        p += pt1 + pt2 + pt3
    return -1*p

alpha=0.01
n=0.0001
beta1=0.0001
beta2=0.5
beta3=0.02
beta4=0.3
beta5=0.1

params=alpha,n,beta1,beta2,beta3,beta4,beta5
solnbdr1 = minimize(
    NBDR,
    args = (a,b,c,d,f,k),
    x0 = np.array(params),
    bounds=[(0.000001,None),(0.000001,None),(None,None),(None,None),(None,None),(None,None),(None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

alpha = solnbdr1.x[0]
n=solnbdr1.x[1]
beta1=solnbdr1.x[2]
beta2=solnbdr1.x[3]
beta3=solnbdr1.x[4]
beta4=solnbdr1.x[5]
beta5=solnbdr1.x[6]
ll_nbdr1 = solnbdr1.fun

print("Alpha: {}".format(alpha))
print("n: {}".format(n))
print("beta1: {}".format(beta1))
print("Beta2: {}".format(beta2))
print("Beta3: {}".format(beta3))
print("Beta4: {}".format(beta4))
print("Beta5: {}".format(beta5))
print("Log Likelihood: {}".format((-1)*ll_nbdr1))

Alpha: 1.7498866921285627
n: 2.2649021059591434
beta1: -0.2157650356969458
Beta2: 0.1492626869924588
Beta3: -0.17626768965454373
Beta4: 0.014831939706369512
Beta5: 0.029098848354927986
Log Likelihood: -1560.9585966009363


In [19]:
df=pd.read_csv('articles.csv')

a=np.array(df[['female']])
b=np.array(df[['married']])
c=np.array(df[['kids']])
d=np.array(df[['prestige']])
f=np.array(df[['menpubs']])
k=np.array(df[['articles']])

#1. we utilized our NBD Regression function and altered it to include zero inflated cases
def NBDRZI(params,a,b,c,d,f,k):
    alpha, n, beta1, beta2, beta3, beta4, beta5, pi = params
    p = 0
    for i in range(len(a)):
        pt1 = np.log(math.gamma(n+int(k[i]))) -(np.log(math.gamma(n)) + np.log(factorial(int(k[i]))))
        denom = math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i]+beta5*f[i])
        pt2 = n*(np.log(alpha) - np.log(alpha+denom))
        pt3 = int(k[i])*(np.log(denom) - np.log(alpha+denom))
        pt4 = np.log(1-pi)
        if k[i]==0: #this will better predict the number of 0's in the model
            pt1 = math.gamma(n+int(k[i]))/math.gamma(n)*factorial(int(k[i]))
            pt2 = (alpha/(alpha+denom))**n
            pt3 = (denom/(alpha+denom))**int(k[i])
            p += np.log(pt1*pt2*pt3*(1-pi)+pi)
        else:
            p += pt1 + pt2 + pt3 + pt4
    return -1*p

#2. we set a few guesses for each of the parameters
alpha=0.07
n=0.03
beta1=0.0001
beta2=0.6
beta3=0.4
beta4=0.03
beta5=0.005
pi=0.01 #pi is a probability, so we set it between 0 and 1, noninclusive
params=alpha,n,beta1,beta2,beta3,beta4,beta5,pi

#3. we optimize the parameters to maximize ll 
solnbdrzi = minimize(
    NBDRZI,
    args = (a,b,c,d,f,k),
    x0 = np.array(params),
    bounds=[(0.000001,None),(0.000001,None),(None,None),(None,None),(None,None),(None,None),(None,None),(0.0000001,1)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

alpha = solnbdrzi.x[0]
n=solnbdrzi.x[1]
beta1=solnbdrzi.x[2]
beta2=solnbdrzi.x[3]
beta3=solnbdrzi.x[4]
beta4=solnbdrzi.x[5]
beta5=solnbdrzi.x[6]
pi=solnbdrzi.x[7]
ll_nbdrzi = solnbdrzi.fun

print(solnbdrzi)

print("Alpha: {}".format(alpha))
print("n: {}".format(n))
print("beta1: {}".format(beta1))
print("Beta2: {}".format(beta2))
print("Beta3: {}".format(beta3))
print("Beta4: {}".format(beta4))
print("Beta:{}".format(beta5))
print("Pi: {}".format(pi))
print("Log Likelihood: {}".format((-1)*ll_nbdrzi))

      fun: 1560.958344994616
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.27556633e-02,  1.31421985e-02,  1.72576620e-02,  2.42152964e-02,
        2.26009434e-02,  5.05679054e-02,  1.34923539e-01,  4.90300863e+01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 396
      nit: 40
     njev: 44
   status: 0
  success: True
        x: array([ 1.75288256e+00,  2.26466627e+00, -2.16362630e-01,  1.50532992e-01,
       -1.76391269e-01,  1.52537849e-02,  2.90817386e-02,  1.00000000e-07])
Alpha: 1.7528825636741736
n: 2.2646662723139137
beta1: -0.2163626299473107
Beta2: 0.1505329924647206
Beta3: -0.1763912690587247
Beta4: 0.015253784915255609
Beta:0.029081738550683838
Pi: 1e-07
Log Likelihood: -1560.958344994616
